<a href=https://experimental-design.github.io/bofire/>
  <img width="350" src="https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/logos/bofire-long.png" alt="BoFire Logo" />
</a>

## Getting started with BoFire

In our [docs](https://experimental-design.github.io/bofire/), you can find all different options for the [BoFire installation](https://experimental-design.github.io/bofire/install/). For basic BoFire Bayesian optimization features using [BoTorch](https://botorch.org/) which depends on
[PyTorch](https://pytorch.org/), you need to run

```
pip install bofire[optimization]
```

For a more complete introduction to BoFire, please look in our [docs](https://experimental-design.github.io/bofire/).

### Optimization problem

You will find a notebook covering the described example below in our [tutorials](https://github.com/experimental-design/bofire/tree/main/tutorials) section to run the code yourself.

Let us consider a test function for single-objective optimization - the [Himmelblau's function](https://en.wikipedia.org/wiki/Himmelblau%27s_function). The Himmelblau's function is a multi-modal function with four identical local minima used to test the performance of optimization algorithms. Let's define the Himmelblau's function to evaluate points in the domain space.

In [2]:
def himmelblau(x1, x2):
    return (x1**2 + x2 - 11) ** 2 + (x1 + x2**2 - 7) ** 2

The optimization domain of the Himmelblau's function is illustrated below together with the four minima marked red.

In [3]:
import numpy as np


x1_vec = np.linspace(-5, 5, 100)
x2_vec = np.linspace(-5, 5, 100)

x_1, x_2 = np.meshgrid(x1_vec, x2_vec)
results = himmelblau(x_1, x_2)

x_minima = [
    [3.0, 2.0],
    [-2.805118, 3.131312],
    [-3.779310, -3.283186],
    [3.584428, -1.848126],
]

In [4]:
import plotly.graph_objects as go


fig = go.Figure(
    data=go.Contour(
        z=results,
        zmin=0,
        zmax=600,
        x=x1_vec,
        y=x2_vec,
        contours_coloring="heatmap",
        colorbar={"title": "y"},
    )
)

fig.update_layout(
    title="Himmelblau's Function",
    autosize=False,
    width=400,
    height=350,
    margin={"l": 10, "r": 10, "b": 10, "t": 40},
    xaxis_title="x1",
    yaxis_title="x2",
    legend_title="y",
)

for x1, x2 in x_minima:
    fig.add_trace(
        go.Scatter(
            x=[x1],
            y=[x2],
            mode="markers",
            showlegend=False,
            marker={"size": 10, "color": "red"},
        )
    )
fig.show()

### Defining the optimization output

Let's consider the single continuous output variable *y* of the Himmelblau's function with the objective to minimize it. In BoFire's terminology, we create a `MinimizeObjective` object to define the optimization objective of a `Continuous Output` feature.

In [5]:
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MinimizeObjective


objective = MinimizeObjective()
output_feature = ContinuousOutput(key="y", objective=objective)

For more details on `Output` features and `Objective` objects, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining the optimization inputs

For the two continuous input variables of the Himmelblau's function *x1* and *x2*, we create two `ContinuousInput` features including boundaries following BoFire's terminology.

In [6]:
from bofire.data_models.features.api import ContinuousInput


input_feature_1 = ContinuousInput(key="x1", bounds=(-5, 5))
input_feature_2 = ContinuousInput(key="x2", bounds=(-5, 5))

For more details on `Input` features, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining the optimization domain

In BoFire's terminology, `Domain` objects fully describe the search space of the optimization problem. `Input` and `Output` features are optionally bound with `Constraint` objects to specify allowed relationships between the parameters. Here, we will run an unconstrained optimization. For more details, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


In [7]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs


domain = Domain(
    inputs=Inputs(features=[input_feature_1, input_feature_2]),
    outputs=Outputs(features=[output_feature]),
)

### Draw candidates and execute experiments

To initialize an iterative Bayesian optimization loop, let's first randomly draw 10 samples from the domain. In BoFire's terminology, those suggested samples are called `Candidates`.

In [8]:
candidates = domain.inputs.sample(10, seed=13)

print(candidates)

         x1        x2
0  1.059211  1.374497
1 -4.176967 -1.589341
2 -3.784766  4.674178
3 -2.449681  1.128971
4  3.055689 -2.594837
5 -4.779897  3.157958
6 -0.040224 -2.291381
7  3.854749  3.860476
8 -0.896600  1.240756
9  0.536098 -0.041359


Let's execute the randomly drawn candidates using the `himmelblau` function to obtain `Experiments` in BoFire's terminology.

In [9]:
experimental_output = candidates.apply(lambda row: himmelblau(row["x1"], row["x2"]), axis=1)

experiments = candidates.copy()
experiments["y"] = experimental_output

print(experiments)

         x1        x2           y
0  1.059211  1.374497   88.725854
1 -4.176967 -1.589341   98.436483
2 -3.784766  4.674178  186.371901
3 -2.449681  1.128971   81.809936
4  3.055689 -2.594837   25.904959
5 -4.779897  3.157958  228.427101
6 -0.040224 -2.291381  179.821177
7  3.854749  3.860476  197.842843
8 -0.896600  1.240756  120.611344
9  0.536098 -0.041359  157.407519


For more details on candidates and experiments, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining an optimization strategy

Let's specify the strategy how the Bayesian optimization campaign should be conducted. Here, we define a single-objective Bayesian optimization strategy and pass the optimization domain together with a acquisition function. Here, we use logarithmic expected improvement `qLogEI` as the acquisition function. In BoFire's terminology, we create a serializable data model `SoboStrategy` which we then map to our functional model.


In [10]:
import bofire.strategies.api as strategies
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.strategies.api import SoboStrategy


sobo_strategy_data_model = SoboStrategy(domain=domain, acquisition_function=qLogEI(), seed=19)

sobo_strategy = strategies.map(sobo_strategy_data_model)

For more details on strategy data models and functional models, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Run the optimization loop

To run the optimization loop using BoFire's terminology, we first `tell` the strategy object about the experiments we have already executed.


In [11]:
sobo_strategy.tell(experiments=experiments)

Subsequently, we run the optimization loop with a budget of 30 iterations. In each iteration, we `ask` the strategy object for one new candidate (returned as a list with one item). Then, we execute the suggested candidate to obtain a new experiment. To complete one full iteration, we concatenate the new experiment to the existing experiments and `tell` the strategy about the updated experiments.

In [ ]:
import pandas as pd


for _ in range(30):
    new_candidates = sobo_strategy.ask(candidate_count=1)

    new_experiments = new_candidates.copy()
    new_experiments["y"] = new_candidates.apply(lambda row: himmelblau(row["x1"], row["x2"]), axis=1)
    experiments = pd.concat([experiments, new_experiments], join="inner").reset_index(drop=True)

    sobo_strategy.tell(experiments=experiments)

The optimization behavior of the strategy is shown in the animated figure below. The four minima are marked red, the experiments carried out are marked blue with blue lines connecting them. The contours are indicating the predicted mean of the current model of each iteration.

In [26]:
# TODO: Add Dorofee's updated plotting function